$$
E_{07} = 1\cdot P_{07} + \sum_{k=1}^{6} (1+E_{k7})\cdot P_{0k} \\
\vdots \\
E_{i7} = 1\cdot P_{i7} + \sum_{\substack{k=0 \\ k\neq i}}^{6} (1+E_{k7})\cdot P_{ik} \\
\vdots \\
E_{67} = 1\cdot P_{67} + \sum_{k=0}^{5} (1+E_{k7})\cdot P_{6k}
$$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import operator as op
from functools import reduce

def comb(n, k):
    k = min(k, n-k)
    num = reduce(op.mul, range(n, n-k, -1), 1)
    den = reduce(op.mul, range(1, k+1), 1)
    return num // den

In [3]:
def bit_diff(a, b):
    x, c = a ^ b, 0
    while x > 0:
        c += 1
        x -= x & -x
    return c

In [4]:
def get_dist(u, v):
    return np.sqrt(bit_diff(u, v))

In [5]:
def get_prob_const(dim):
    return 1 / sum(1/np.sqrt(i) * comb(dim, i) for i in range(1, dim+1))

In [6]:
def get_prob(u, v, C):
    if u == v: return 0
    return C / get_dist(u, v)

In [7]:
def get_prob_matrix(n_vertices, C):
    return np.array([[get_prob(u, v, C) for v in range(n_vertices)] for u in range(n_vertices)])

In [8]:
def update_E(E, P, n_vertices):
    for u in range(n_vertices-1):
        E[u] = P[u, n_vertices-1]
        for v in range(n_vertices-1):
            if u == v: continue
            E[u] += (1 + E[v]) * P[u, v]
    return E

In [9]:
def run_epochs(E, n_iters, P, n_vertices, Es=None):
    for _ in range(n_iters):
        E = update_E(E, P, n_vertices)
        if Es: Es.append(E[0])
    return E

In [10]:
def solve(dim, n_iter=3000):
    n_vertices = 2**dim
    C = get_prob_const(dim)
    P = get_prob_matrix(n_vertices, C)
    E, Es = np.zeros(n_vertices-1), [0]
    E = run_epochs(E, n_iter, P, n_vertices, Es)
    return E[0], Es, E

In [11]:
E3, *_ = solve(3)
E3

7.30772726461206

In [12]:
E7, *_ = solve(7)
E7

127.43899470063619